# Introduction to Bayesian Additive Regression Trees
Bayesian Additive Regression Trees (BART) is a flexible prediction model that uses a collection of decision trees, each contributing uniquely to the final outcome. It doesn't assume a specific pattern in the data, allowing it to learn directly from the information it receives. One advantage is that it prevents overfitting, which means that it can make accurate predictions on new databy generalizing well from what it has learnt.

## 1 | Necessary Imports
The cell below shows the necessary imports of several Python libraries that are commonly used for statistical modeling and data visualization. Here is a short description of each of them as follows:
- **ArviZ (az):** A library for exploratory analysis of Bayesian models.
- **Matplotlib (plt):** A plotting library for creating static, animated, and interactive visualizations in Python.
- **NumPy (np):** A library for numerical computing, providing support for arrays and mathematical functions.
- **Pandas (pd):** A data manipulation library that offers data structures for efficiently storing and analyzing data.
- **PyMC (pm):** A library for probabilistic programming that allows users to build Bayesian models.
- **PyMC-BART (pmb):** An extension for PyMC that implements Bayesian Additive Regression Trees (BART).
- **PyTensor (pt):** A library for symbolic computation used for defining and evaluating mathematical expressions.

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import pymc_bart as pmb
import pytensor.tensor as pt

## 2 | Visual Styling & Color Cycles
The cell below is used to establish and configure a visual style for the plots created using Matplotlib and sets up a color cycle for the axes

In [ ]:
# applies grayscale color scheme to ArviZ plots
az.style.use("arviz-grayscale")

# importing the cycler class
from cycler import cycler

# creates a custom color cycle using four specific colors as defined with the hex codes
default_cycler = cycler(color=["#000000", "#6a6a6a", "#bebebe", "#2a2eec"])

# sets the default color cycle for the axes and the default resolution
plt.rc("axes", prop_cycle=default_cycler)
plt.rc("figure", dpi=300)

# random seed to ensure any random numbers generated in session is reproducible
np.random.seed(123)

## 3 | Data Preprocessing
The cell below performs data preprocessing on a dataset of penguins, which is typically used for regression or classification tasks.

In [ ]:
# loading the .csv file 
pen = pd.read_csv("penguins.csv").dropna()

# setting the target variable (body mass), which is the variable for prediction
Y = pen["body_mass"]

# replacing categorical values in each column with numerical values
pen['sex'].replace(['male', 'female'],[0, 1], inplace=True)
pen['species'].replace(['Adelie', 'Chinstrap', 'Gentoo'],[0, 1, 2], inplace=True)

# creating a new dataframe X which contains the feature variables for modeling
X = pen[["flipper_length", "bill_depth", "bill_length"]]

## 4 | Bayesian Regression Model

The cell below shows how to build a Bayesian Regression Model using PyMC and Bayesian Additive Regression Trees (BART) to analyze the penguin dataset.

`σ = pm.HalfNormal("σ", 1)` is used to represent the **standard deviation** in the model. 
- The half-normal distribution means we only consider positive values for the standard deviation.
- The parameter `1` controls the range of possible values for `σ`.
- `"σ"` is a string that identifies the distribution in the model's output.

`μ = pmb.BART("μ", X, Y, m=50)` is used to create a BART model that predicts the mean `μ` of the target variable `Y` based on the input features `X`. 
- `"μ"` is the name of the mean variable in the model.
- `m` is the number of decision trees used to make predictions.

`y = pm.Normal("y", mu=μ, sigma=σ, observed=Y)` defines how the observed data `Y` relates to the predicted mean `μ` and the standard deviation `σ`.
- `y` defines the likelihood variable in the model.
- `mu=μ` specifies the predicted mean that comes from the BART model.
- `sigma=σ` indicates the variability in the predictions controlled by the standard deviation we set.
- `observed=Y` links the actual observed data (body m, allowing it to learn. the model learned.
viation `σ`.




### Code Explanation
This is a flexible model that learns from past data (body masses of penguins) and allows for making educated guesses about new data (predicting body mass for other penguins). It captures the relationships in the data, accounts for uncertainty, and allows us to see a range of possible values based on what the model learned.

In [ ]:
# creating a probabilistic model context using PyMC
with pm.Model() as model_pen:
    σ = pm.HalfNormal("σ", 1)
    μ = pmb.BART("μ", X, Y, m=50)
    y = pm.Normal("y", mu=μ, sigma=σ, observed=Y)

    # sampling process to estimate parameters of the model based on observed data
    idata_pen = pm.sample(random_seed=123)

    # generates new data based on model's learned parameters to see how well it can predict new observations
    pm.sample_posterior_predictive(idata_pen, extend_inferencedata=True, random_seed=123)

In [ ]:
# retrieving the smallest predicted value of the outcome based on the built model
idata_pen.posterior_predictive.min()

## 5 | Plotting the Model
The cell below is used to create a plot of the posterior predictive checks (PPC). PPC is important because it helps with assessing how well the Bayesian Model fits the observed data. If the PPC don't align well with the observed data, this indicates that the model isn't adequately capturing relationships in the data.

`ax = az.plot_ppc(idata_pen, num_pp_samples=100, colors=["C1", "C0", "C1"])` is used to create a plot of the posterior predictive checks (PPC). 
- `az.plot_ppc()` is used to generate a posterior predictive check plot, which visualizes how well the model's predictions match observed data
- `num_pp_samples` specifies the number of posterior predictive samples used for plotting
- `colors=["C1", "C0", "C1"]` used to set the colors for the plotted data


In [ ]:
ax = az.plot_ppc(idata_pen, num_pp_samples=100, colors=["C1", "C0", "C1"])
ax.set_xlabel("body_mass (kg)")
plt.savefig("BART_ppc.png")

In [ ]:
# generates a summary of the posterior distribution of the variable μ from object idata_pen
az.summary(idata_pen, var_names=["~μ"])

## 6 | Partial Dependence Plotting
The cell below creates a visual representation of how each input feature affects the predicted body mass of penguins based on the BART model. Each subplot in the figure corresponds to one of the input features, showing expected changes in the predicted outcome as that feature varies.

`pmb.plot_pdp(μ, X, Y, grid=(1, 3), figsize=(10, 4))` 
- `pmb.plot_pdp` generates partial dependence plots, which visualizes the relationship between predicted outcome and one or more input features
- `μ` is the mean prediction of the BART model, `X` is the input features and `Y` is the target variable


In [ ]:
pmb.plot_pdp(μ, X, Y, grid=(1, 3), figsize=(10, 4))
plt.savefig("BART_pdp.png", bbox_inches="tight")

## 7 | Individual Conditional Expectation (ICE) Plot
The cell below generates a Individual Conditional Expectation (ICE) plot using the `pymc_bart` library. We create a series of ICE plots that show individual predictions from the BART model change as we vary one feature while keeping the other fixed. Each line in the plots represent the prediction for a specific observation as the feature of interest changes.

In [ ]:
pmb.plot_ice(μ, X, Y, grid=(1, 3), figsize=(10, 4))
plt.savefig("BART_ice.png", bbox_inches="tight")

## 8 | Variable Importance
The cell below generates and saves a plot of variable importance for the Bayesian Additive Regression Trees (BART) model. The plots visually shows the relative importance of each input feature in predicting the target variable (body mass) based on the BART model.

In [ ]:
# used to plot the variable importance of each feature in the model
_, axes = pmb.plot_variable_importance(idata_pen, μ, X, figsize=(10, 6))

# sets the y-axis range
axes[0].set_ylim(0, .6)   # first subplot
axes[1].set_ylim(0.7, 1)  # second subplot

# saving diagram as a figure
plt.savefig("BART_VI.png")

## 9 | Convergence Diagnostic Plot
The cell below generates and saves a converge diagnostic plot for the BART model. The code will produce a plot that helps you visually check if sampling chains for the μ have reached a stable distribution. A converged chain is important because it indicates that the model has explored posterior distribution, and the results are reliable.

### Effective Sample Size (ESS) vs R-hat (Gelman-Rubin Statistic)
**ESS:** measures the number of effectively independent samples in the chain. Due to MCMC samples being correlated, ESS is often smaller than the total number of samples. Larger the ESS, more reliable the estimate because it suggests the parameter has been explored

**R-hat:** a diagnostic metric that measures the convergence of MCMC chains. Compares the variance within each chain to the variance between multiple chains to check if they're sampling from the same distribution. Values below 1.01 are a sign of strong convergence.

In [ ]:
# used to plot the converge diagnostic plot using idata_pen
pmb.plot_convergence(idata_pen, var_name="μ", figsize=(10, 4))

# saves the figure
plt.savefig("bart_plot_convergence.png")

## 10 | Constant vs linear responses
The cell below sets up `X` as a feature and `Y` as the target variable for further analysis. The reshaping of `X` ensures that it's ready for models expecting a 2D feature input.

In [ ]:
bikes = pd.read_csv("bikes.csv")
X = bikes["temperature"].values[:,None]
Y = bikes["rented"]

## 11 | Bayesian Regression Model with Mutable Input Data
The cell below defines a Bayesian regression model to analyze how temperature influences bike renatals. It uses BART to predict the log of the count of rented bikes with a negative binomial distibution for the observed rental data. 

`μ = pmb.BART("μ", X_mut0, np.log(Y), m=100)`
- `X_mut0` is the input feature, which is the temperature
- `m=100` indicates that the BART model will use 100 trees for making predictions

`_ = pm.NegativeBinomial('yl', np.exp(μ), α, observed=Y, shape=μ.shape)`
- `np.exp(μ)` transforms the predicted mean back to the original scale
- `α`is the shape parameter from the half-normal distribution
- `observed=Y` indicates that the actual bike rental counts are the observed data for this distribution
- `shape=μ.shape` ensures that the shape of the negative binoial distribution matches the predictions from the BART model

In [ ]:
with pm.Model() as model_tmp0:

    # creates a mutable data variable that holds the input feature matrix X
    X_mut0 = pm.MutableData("X_mut0", X)

    # defines a half-normal distribution, 1 specifies scale of distribution
    α = pm.HalfNormal('α', 1)

    # creates a BART model that predicts the mean of the logged target variable Y
    μ = pmb.BART("μ", X_mut0, np.log(Y), m=100)
    _ = pm.NegativeBinomial('yl', np.exp(μ), α, observed=Y, shape=μ.shape)

    # runs the sampling processto draw samples from the posterior distribution of the model parameters
    idata_tmp0 = pm.sample(random_seed=123)

### Modeling Linear Relationships
The cell below is the same thing, but BART models the relationship between features and the response variable as a linear function. This allows the BART model to capture linear relationships, potentially simplifying the model.

In [ ]:
with pm.Model() as model_tmp1:
    X_mut1 = pm.MutableData("X_mut1", X)
    α = pm.HalfNormal('α', 1)
    μ = pmb.BART("μ", X_mut1, np.log(Y), m=100, response="linear")
    _ = pm.NegativeBinomial('yl', np.exp(μ), α, observed=Y, shape=μ.shape)
    idata_tmp1 = pm.sample(random_seed=123)

## 12 | Generate Posterior Predictive Samples

The cell below generates posterior predictive samples for bike rentals based on a new set of temperature values using a previously defined Bayesian regression model. 

- `new_x = np.linspace(-20, 45, 100)` creates an array of 100 evenly spaced values between -20 and 45.
- `X_mut0.set_value(new_x[:, None])` updates the mutable data with the new temperature values from `new_x`.
- `pp0 = pm.sample_posterior_predictive(idata_tmp0, random_seed=123)` draws samples from the posterior predictive distribution using the fitted model.

```

In [ ]:
new_x = np.linspace(-20, 45, 100)
with model_tmp0:
    X_mut0.set_value(new_x[:,None])
    pp0 = pm.sample_posterior_predictive(idata_tmp0, random_seed=123)

In [ ]:
with model_tmp1:
    X_mut1.set_value(new_x[:,None])
    pp1 = pm.sample_posterior_predictive(idata_tmp1, random_seed=123)

In [ ]:
# computes the mean predicted values from the posterior predictive samples of the first and second model
pred_y0 = pp0.posterior_predictive.mean(("chain", "draw"))["yl"]
pred_y1 = pp1.posterior_predictive.mean(("chain", "draw"))["yl"]

# initializes a new figure and axes object for the plot with a size of 10 by 4 inches
_, ax = plt.subplots(figsize=(10, 4))

# plots the predicted bike rentals against the new temperature values for the constant response model
ax.plot(new_x, pred_y0, color="C0", lw=3, label="constant response")
ax.plot(new_x, pred_y1, color="C1", lw=3, label="linear response")

# does the dot plotting of the original data of temperature (X) versus bike count (Y)
ax.plot(X, Y, ".", color="C2", zorder=-1)

# adding dashed vertical lines at the minimum and maximum temperature values
ax.axvline(X.min(), color="C2", linestyle="--")
ax.axvline(X.max(), color="C2", linestyle="--")

# does the graph labels and legend
ax.set_xlabel("temperature")
ax.set_ylabel("rented")
plt.legend()
plt.savefig("BART_constant_vs_linear.png")

## 13 | Bayesian Regression Model with Seperate Mean Predictions
In the cell below, we initialize the data for a modeling task by setting a random seed for reproducibility, specifying the features and the target variable needed for analysis. 

In [ ]:
np.random.seed(123)
bikes = pd.read_csv("bikes.csv")
X = bikes[["hour", "temperature", "humidity", "weekday"]]
Y = bikes["rented"]

### Building the Bayesian Regression Model
The cell below builds a bayesian regression model using BART to predict bike rentals based on various features. The approaches mentioned below allows for capturing complex relationships in the data while accounting for overdispersion. It does the following:
- transforms the target variable with a logarithm for better modeling
- defines a Negative Binomial distribution for the observed data
- samples from the model to estimate the posterior distributions of the parameters

`μ = pmb.BART("μ", X, np.log(Y), shape=(2, 348), separate_trees=True)`
- `X` contains the features used for prediction (the independent variables)
- `np.log(Y)` is the nautral logarithm of the target variable, we take log() to stabilize variance and make data more normally distributed
- `shape(2, 348)` specifies shape of model, where there are 2 separate predictions for 348 observations
- `separate_trees=True` indicates that the mean will be modeled using separate trees

`pm.NegativeBinomial('yl', np.exp(μ[0]), np.exp(μ[1]), observed=Y)` where `observed=Y` 
- `np.exp(μ[0])` and `np.exp(μ[1])` compute the mean parameters for the Negative Binomial Distribution by exponentiating the predictions from the model
- `observed=Y` links the observed data to the model, allowing it to learn from the data

`idata_bb = pm.sample(2000, random_seed=123, pgbart={"batch":(0.05, 0.15)})`
- `2000` is the number of samples drawn from the posterior
- `pgbart={"batch":(0.05, 0.15)}` where `batch` controls how many decision trees are used in the sampling process during two different phases of tuning and actual sampling. `0.05` and `0.15` specifies the proportions of trees to be used.

In [ ]:
with pm.Model() as model_bb:

    # predicted mean of the target variable (number of rented bikes)
    μ = pmb.BART("μ", X, np.log(Y), shape=(2, 348), separate_trees=True)

    # defines likelihood of observed data (Y) using a Negative Binominal Distribution
    pm.NegativeBinomial('yl', np.exp(μ[0]), np.exp(μ[1]), observed=Y)

    # 
    idata_bb = pm.sample(2000,
                         random_seed=123,
                         pgbart={"batch":(0.05, 0.15)})

### Creating a Partial Dependence Plot
In the cell below, we generate a series of partial dependence plots for the BART model trained on bike rental data. This visualizes how chnages in each feature impacts the predicted number of bike rentals while controlling other features.

`pmb.plot_pdp(μ, X, Y, xs_interval="insample", grid=(4, 2), sharey="col", figsize=(10, 6), func=np.exp , var_discrete=[3])`
- `xs_interval="insample"` indicates that PDP will be calculated based on the feature values present in the training data
- `func=np.exp` applies the exponential function to the predictions
- `var_discrete=[3]` the variable at index 3 (weekday) is treated as discrete. It means for that variable, the plot will show the average predicted value for each category. 

### Why are there two graphs?
1. **First Graph:** the graph represents the average predicted outcome across the range of a continuous variable, which shows how changing the variable affects the predictions.
2. **Second Graph:** the graph tpyically represents the predictions for a discrete variable. It might display how the predictions vary across different categories of that variable.

In [ ]:
pmb.plot_pdp(μ, X, Y, xs_interval="insample", grid=(4, 2), sharey="col", figsize=(10, 6), func=np.exp , var_discrete=[3])
plt.savefig("BART_pdp_bikes.png", bbox_inches="tight")